# **Stock Prices Analysis using PySpark SQL**

![Sample Image](https://www.sharegurukul.com/wp-content/uploads/2018/12/technical-analysis.jpg)

In [4]:
! pip install pyspark

     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 198 kB 41.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=a6c53104c0c026a8c907b368e66bf4e605889452a8b82a01574a486960fb1c90
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [1]:
import pyspark
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

24/09/30 15:24:54 WARN Utils: Your hostname, Yains-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.48 instead (on interface en0)
24/09/30 15:24:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/30 15:24:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [3]:
type(sc)

pyspark.context.SparkContext

In [4]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)

In [5]:
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

/Users/tequan/miniforge3/lib/python3.10/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [6]:
l = [('Alice', 1)]
df = sqlContext.createDataFrame(l, ['name', 'age'])
df.collect()

[Row(name='Alice', age=1)]

In [9]:
data_file = 'TSLA.csv'
raw_rdd = sc.textFile(data_file).cache()
raw_rdd.take(20)

['Date,Open,High,Low,Close,Adj Close,Volume',
 '2010-06-29,1.266667,1.666667,1.169333,1.592667,1.592667,281494500',
 '2010-06-30,1.719333,2.028000,1.553333,1.588667,1.588667,257806500',
 '2010-07-01,1.666667,1.728000,1.351333,1.464000,1.464000,123282000',
 '2010-07-02,1.533333,1.540000,1.247333,1.280000,1.280000,77097000',
 '2010-07-06,1.333333,1.333333,1.055333,1.074000,1.074000,103003500',
 '2010-07-07,1.093333,1.108667,0.998667,1.053333,1.053333,103825500',
 '2010-07-08,1.076000,1.168000,1.038000,1.164000,1.164000,115671000',
 '2010-07-09,1.172000,1.193333,1.103333,1.160000,1.160000,60759000',
 '2010-07-12,1.196667,1.204667,1.133333,1.136667,1.136667,33037500',
 '2010-07-13,1.159333,1.242667,1.126667,1.209333,1.209333,40201500',
 '2010-07-14,1.196000,1.343333,1.184000,1.322667,1.322667,62928000',
 '2010-07-15,1.329333,1.433333,1.266667,1.326000,1.326000,56097000',
 '2010-07-16,1.380000,1.420000,1.336667,1.376000,1.376000,39319500',
 '2010-07-19,1.424667,1.483333,1.394667,1.460667,1.

In [8]:
type(raw_rdd)

pyspark.rdd.RDD

In [10]:
csv_rdd = raw_rdd.map(lambda row: row.split(","))
print(csv_rdd.take(2))
print(type(csv_rdd))

[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], ['2010-06-29', '1.266667', '1.666667', '1.169333', '1.592667', '1.592667', '281494500']]
<class 'pyspark.rdd.PipelinedRDD'>


In [11]:
len(csv_rdd.take(1)[0])

7

In [12]:
df = sqlContext.read.load(data_file, 
                      format='com.databricks.spark.csv', 
                      header='true', 
                      inferSchema='true')

In [13]:
df.count()

3562

In [14]:
df.take(5)

[Row(Date=datetime.date(2010, 6, 29), Open=1.266667, High=1.666667, Low=1.169333, Close=1.592667, Adj Close=1.592667, Volume=281494500),
 Row(Date=datetime.date(2010, 6, 30), Open=1.719333, High=2.028, Low=1.553333, Close=1.588667, Adj Close=1.588667, Volume=257806500),
 Row(Date=datetime.date(2010, 7, 1), Open=1.666667, High=1.728, Low=1.351333, Close=1.464, Adj Close=1.464, Volume=123282000),
 Row(Date=datetime.date(2010, 7, 2), Open=1.533333, High=1.54, Low=1.247333, Close=1.28, Adj Close=1.28, Volume=77097000),
 Row(Date=datetime.date(2010, 7, 6), Open=1.333333, High=1.333333, Low=1.055333, Close=1.074, Adj Close=1.074, Volume=103003500)]

In [15]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: integer (nullable = true)



In [16]:
import pandas
df.toPandas().head(5)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,1.266667,1.666667,1.169333,1.592667,1.592667,281494500
1,2010-06-30,1.719333,2.028000,1.553333,1.588667,1.588667,257806500
2,2010-07-01,1.666667,1.728000,1.351333,1.464000,1.464000,123282000
3,2010-07-02,1.533333,1.540000,1.247333,1.280000,1.280000,77097000
4,2010-07-06,1.333333,1.333333,1.055333,1.074000,1.074000,103003500


In [25]:
from pyspark.sql.types import *
schema = StructType([
   StructField("Date", DateType(), True),
   StructField("Open", DoubleType(), True),
   StructField("High", DoubleType(), True),
   StructField("Low", DoubleType(), True),
   StructField("Close", DoubleType(), True),
   StructField("Adj Close", DoubleType(), True),
   StructField("Volume", IntegerType(), True)])


df2 = sqlContext.read.load(data_file, 
                      format='com.databricks.spark.csv', 
                      header='true', 
                      schema=schema)

In [26]:
df2.take(5)

[Row(Date=datetime.date(2010, 6, 29), Open=1.266667, High=1.666667, Low=1.169333, Close=1.592667, Adj Close=1.592667, Volume=281494500),
 Row(Date=datetime.date(2010, 6, 30), Open=1.719333, High=2.028, Low=1.553333, Close=1.588667, Adj Close=1.588667, Volume=257806500),
 Row(Date=datetime.date(2010, 7, 1), Open=1.666667, High=1.728, Low=1.351333, Close=1.464, Adj Close=1.464, Volume=123282000),
 Row(Date=datetime.date(2010, 7, 2), Open=1.533333, High=1.54, Low=1.247333, Close=1.28, Adj Close=1.28, Volume=77097000),
 Row(Date=datetime.date(2010, 7, 6), Open=1.333333, High=1.333333, Low=1.055333, Close=1.074, Adj Close=1.074, Volume=103003500)]

In [27]:
df2.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: integer (nullable = true)



In [28]:
type(df)

pyspark.sql.dataframe.DataFrame

In [29]:
df.show(5)

+----------+--------+--------+--------+--------+---------+---------+
|      Date|    Open|    High|     Low|   Close|Adj Close|   Volume|
+----------+--------+--------+--------+--------+---------+---------+
|2010-06-29|1.266667|1.666667|1.169333|1.592667| 1.592667|281494500|
|2010-06-30|1.719333|   2.028|1.553333|1.588667| 1.588667|257806500|
|2010-07-01|1.666667|   1.728|1.351333|   1.464|    1.464|123282000|
|2010-07-02|1.533333|    1.54|1.247333|    1.28|     1.28| 77097000|
|2010-07-06|1.333333|1.333333|1.055333|   1.074|    1.074|103003500|
+----------+--------+--------+--------+--------+---------+---------+
only showing top 5 rows



In [30]:
tesla_data_file = 'TSLA.csv'

tesla_df = sqlContext.read.load(tesla_data_file, 
                      format='com.databricks.spark.csv', 
                      header='true', 
                      inferSchema='true')

tesla_df.show()

+----------+--------+--------+--------+--------+---------+---------+
|      Date|    Open|    High|     Low|   Close|Adj Close|   Volume|
+----------+--------+--------+--------+--------+---------+---------+
|2010-06-29|1.266667|1.666667|1.169333|1.592667| 1.592667|281494500|
|2010-06-30|1.719333|   2.028|1.553333|1.588667| 1.588667|257806500|
|2010-07-01|1.666667|   1.728|1.351333|   1.464|    1.464|123282000|
|2010-07-02|1.533333|    1.54|1.247333|    1.28|     1.28| 77097000|
|2010-07-06|1.333333|1.333333|1.055333|   1.074|    1.074|103003500|
|2010-07-07|1.093333|1.108667|0.998667|1.053333| 1.053333|103825500|
|2010-07-08|   1.076|   1.168|   1.038|   1.164|    1.164|115671000|
|2010-07-09|   1.172|1.193333|1.103333|    1.16|     1.16| 60759000|
|2010-07-12|1.196667|1.204667|1.133333|1.136667| 1.136667| 33037500|
|2010-07-13|1.159333|1.242667|1.126667|1.209333| 1.209333| 40201500|
|2010-07-14|   1.196|1.343333|   1.184|1.322667| 1.322667| 62928000|
|2010-07-15|1.329333|1.433333|1.26

In [67]:
google_data_file = 'GOOG.csv'

google_df = sqlContext.read.load(google_data_file, 
                      format='com.databricks.spark.csv', 
                      header='true', 
                      inferSchema='true')

google_df.show()

+----------+---------+---------+---------+---------+---------+--------+
|      Date|     Open|     High|      Low|    Close|Adj Close|  Volume|
+----------+---------+---------+---------+---------+---------+--------+
|2004-08-19|50.050049|52.082081|48.028027|50.220219|50.220219|44659096|
|2004-08-20|50.555557|54.594597|50.300301|54.209209|54.209209|22834343|
|2004-08-23|55.430431|56.796799|54.579578|54.754753|54.754753|18256126|
|2004-08-24|55.675674|55.855858|51.836838|52.487488|52.487488|15247337|
|2004-08-25|52.532532|54.054054|51.991993|53.053055|53.053055| 9188602|
|2004-08-26|52.527527| 54.02903|52.382381| 54.00901| 54.00901| 7094898|
|2004-08-27|54.104103|54.364365|  52.8979|53.128128|53.128128| 6211782|
|2004-08-30|52.692692|52.797798|51.056057|51.056057|51.056057| 5196798|
|2004-08-31|51.201202|51.906906| 51.13113|51.236237|51.236237| 4917877|
|2004-09-01|51.401402|51.536537|49.884884|50.175175|50.175175| 9138253|
|2004-09-02|49.644646|51.236237| 49.51952|50.805805|50.805805|15

In [32]:
amazon_data_file = 'AMZN.csv'

amazon_df = sqlContext.read.load(amazon_data_file, 
                      format='com.databricks.spark.csv', 
                      header='true', 
                      inferSchema='true')

amazon_df.show()

+----------+--------+--------+--------+--------+---------+----------+
|      Date|    Open|    High|     Low|   Close|Adj Close|    Volume|
+----------+--------+--------+--------+--------+---------+----------+
|1997-05-15|0.121875|   0.125|0.096354|0.097917| 0.097917|1443120000|
|1997-05-16|0.098438|0.098958|0.085417|0.086458| 0.086458| 294000000|
|1997-05-19|0.088021|0.088542| 0.08125|0.085417| 0.085417| 122136000|
|1997-05-20|0.086458|  0.0875|0.081771|0.081771| 0.081771| 109344000|
|1997-05-21|0.081771|0.082292| 0.06875|0.071354| 0.071354| 377064000|
|1997-05-22|0.071875|0.072396|0.065625|0.069792| 0.069792| 235536000|
|1997-05-23|0.070313|0.076042|0.066667|   0.075|    0.075| 318744000|
|1997-05-27|0.075521|0.082292|0.072917|0.079167| 0.079167| 173952000|
|1997-05-28| 0.08125|0.081771|0.076563|0.076563| 0.076563|  91488000|
|1997-05-29|0.077083|0.077083|0.073958| 0.07526|  0.07526|  69456000|
|1997-05-30|   0.075|0.075521|0.073958|   0.075|    0.075|  51888000|
|1997-06-02|0.075521

In [35]:
import datetime
from pyspark.sql.functions import year, month, dayofmonth

# Average closing price per year for AMZN
# stocksDF.select(year($"dt").alias("yr"), $"adjcloseprice").groupBy("yr").avg("adjcloseprice").orderBy(desc("yr")).show

amazon_df.select(year("Date").alias("year"), "Adj Close").groupby("year").avg("Adj Close").sort("year").show()

+----+-------------------+
|year|     avg(Adj Close)|
+----+-------------------+
|1997|0.15632001250000002|
|1998| 0.7830556865079368|
|1999|  3.453115825396825|
|2000| 2.3266834087301578|
|2001| 0.6100212943548382|
|2002| 0.8242063492063496|
|2003| 1.8870694444444442|
|2004| 2.1782619047619067|
|2005| 1.9948551587301593|
|2006| 1.7954661354581667|
|2007| 3.3614741035856577|
|2008|  3.494001976284587|
|2009|  4.364091269841268|
|2010|  6.957019841269843|
|2011|  9.833601190476193|
|2012| 11.015182000000001|
|2013| 14.901579400793652|
|2014| 16.627551603174613|
|2015| 23.906914666666676|
|2016|  34.97615664285715|
+----+-------------------+
only showing top 20 rows



In [36]:
# Compute the average closing price per month for apc
# stocksDF.select(year($"dt").alias("yr"),month($"dt").alias("mo"), $"adjcloseprice")
# .groupBy("yr","mo").agg(avg("adjcloseprice")).orderBy(desc("yr"),desc("mo")).show

amazon_df.select(year("Date").alias("year"),
                month("Date").alias("month"),
                "Adj Close").groupby("year", "month").avg("Adj Close").sort("year", "month").show()

+----+-----+-------------------+
|year|month|     avg(Adj Close)|
+----+-----+-------------------+
|1997|    5|0.07942718181818181|
|1997|    6|0.07688490476190475|
|1997|    7|0.11027468181818184|
|1997|    8|0.11156990476190473|
|1997|    9|0.17462795238095238|
|1997|   10| 0.2133718695652173|
|1997|   11|0.21892821052631578|
|1997|   12|0.22816059090909088|
|1998|    1|0.24019535000000003|
|1998|    2| 0.2648712105263158|
|1998|    3|0.33668636363636356|
|1998|    4| 0.3816965714285714|
|1998|    5| 0.3737306000000001|
|1998|    6| 0.5772728181818182|
|1998|    7| 0.9896070909090908|
|1998|    8| 0.9977245714285716|
|1998|    9|  0.750328761904762|
|1998|   10| 0.8834576363636365|
|1998|   11| 1.2950000000000002|
|1998|   12| 2.1854403181818185|
+----+-----+-------------------+
only showing top 20 rows



In [37]:
amazon_df.select(year("Date").alias("year"),
                month("Date").alias("month"),
                "Adj Close").groupby("year", "month").avg("Adj Close").sort("year", "month").explain(True)

== Parsed Logical Plan ==
'Sort ['year ASC NULLS FIRST, 'month ASC NULLS FIRST], true
+- Aggregate [year#527, month#528], [year#527, month#528, avg(Adj Close#411) AS avg(Adj Close)#536]
   +- Project [year(Date#406) AS year#527, month(Date#406) AS month#528, Adj Close#411]
      +- Relation [Date#406,Open#407,High#408,Low#409,Close#410,Adj Close#411,Volume#412] csv

== Analyzed Logical Plan ==
year: int, month: int, avg(Adj Close): double
Sort [year#527 ASC NULLS FIRST, month#528 ASC NULLS FIRST], true
+- Aggregate [year#527, month#528], [year#527, month#528, avg(Adj Close#411) AS avg(Adj Close)#536]
   +- Project [year(Date#406) AS year#527, month(Date#406) AS month#528, Adj Close#411]
      +- Relation [Date#406,Open#407,High#408,Low#409,Close#410,Adj Close#411,Volume#412] csv

== Optimized Logical Plan ==
Sort [year#527 ASC NULLS FIRST, month#528 ASC NULLS FIRST], true
+- Aggregate [year#527, month#528], [year#527, month#528, avg(Adj Close#411) AS avg(Adj Close)#536]
   +- Project [

In [68]:
# register the DataFrames as temp views

amazon_df.createOrReplaceTempView("amazon_stocks")
google_df.createOrReplaceTempView("google_stocks")
tesla_df.createOrReplaceTempView("tesla_stocks")

In [69]:
sqlContext.sql("SELECT * FROM amazon_stocks").show(5)

+----------+--------+--------+--------+--------+---------+----------+
|      Date|    Open|    High|     Low|   Close|Adj Close|    Volume|
+----------+--------+--------+--------+--------+---------+----------+
|1997-05-15|0.121875|   0.125|0.096354|0.097917| 0.097917|1443120000|
|1997-05-16|0.098438|0.098958|0.085417|0.086458| 0.086458| 294000000|
|1997-05-19|0.088021|0.088542| 0.08125|0.085417| 0.085417| 122136000|
|1997-05-20|0.086458|  0.0875|0.081771|0.081771| 0.081771| 109344000|
|1997-05-21|0.081771|0.082292| 0.06875|0.071354| 0.071354| 377064000|
+----------+--------+--------+--------+--------+---------+----------+
only showing top 5 rows



In [70]:
# Calculate and display the average closing price per month for XOM ordered by year,month 

sqlContext.sql("""
SELECT 
    year(amazon_stocks.Date) as yr, 
    month(amazon_stocks.Date) as mo, 
    avg(amazon_stocks.`Adj Close`) as avg_adj_close
FROM 
    amazon_stocks 
GROUP BY 
    year(amazon_stocks.Date), 
    month(amazon_stocks.Date)
ORDER BY yr, mo
""").show()




+----+---+-------------------+
|  yr| mo|      avg_adj_close|
+----+---+-------------------+
|1997|  5|0.07942718181818181|
|1997|  6|0.07688490476190475|
|1997|  7|0.11027468181818184|
|1997|  8|0.11156990476190473|
|1997|  9|0.17462795238095238|
|1997| 10| 0.2133718695652173|
|1997| 11|0.21892821052631578|
|1997| 12|0.22816059090909088|
|1998|  1|0.24019535000000003|
|1998|  2| 0.2648712105263158|
|1998|  3|0.33668636363636356|
|1998|  4| 0.3816965714285714|
|1998|  5| 0.3737306000000001|
|1998|  6| 0.5772728181818182|
|1998|  7| 0.9896070909090908|
|1998|  8| 0.9977245714285716|
|1998|  9|  0.750328761904762|
|1998| 10| 0.8834576363636365|
|1998| 11| 1.2950000000000002|
|1998| 12| 2.1854403181818185|
+----+---+-------------------+
only showing top 20 rows



In [71]:
# When did the closing price for SPY go up or down by more than 2 dollars?

sqlContext.sql("""
               SELECT 
                    google_stocks.Date, 
                    google_stocks.Open, 
                    google_stocks.Close, 
                    abs(google_stocks.Close - google_stocks.Open) as spydif 
               FROM google_stocks 
               WHERE abs(google_stocks.Close - google_stocks.Open) > 4 
               """).show()



+----------+----------+----------+------------------+
|      Date|      Open|     Close|            spydif|
+----------+----------+----------+------------------+
|2004-10-25| 88.288284| 93.793793| 5.505508999999989|
|2004-10-29| 99.544548| 95.415413| 4.129135000000005|
|2004-11-05| 91.081078| 84.759758|           6.32132|
|2004-11-11| 84.649651| 91.601601|6.9519499999999965|
|2005-01-26| 89.724724| 94.714714| 4.989990000000006|
|2005-02-02|107.882881|103.083084| 4.799796999999998|
|2005-02-07|102.732735| 98.113113| 4.619622000000007|
|2005-02-09|100.480476| 95.885887| 4.594588999999999|
|2005-02-14| 91.516518| 96.591591| 5.075072999999989|
|2005-05-23|121.701698|127.852852| 6.151154000000005|
|2005-05-25|126.491493|130.535538| 4.044044999999997|
|2005-06-06|141.336334|145.615616| 4.279281999999995|
|2005-06-08|146.571579|139.919922|          6.651657|
|2005-06-20|138.183182|  143.4935| 5.310318000000024|
|2005-06-29|151.401398|  146.5065| 4.894898000000012|
|2005-09-07|143.088089|147.5

In [72]:
#What was the max, min closing price for SPY and XOM by Year?</font> 

sqlContext.sql("""
               SELECT 
                    year(tesla_stocks.Date) as yr, 
                    max(tesla_stocks.`Adj Close`), 
                    min(tesla_stocks.`Adj Close`) 
               FROM tesla_stocks 
               group By year(tesla_stocks.Date)
               order by yr
               """).show()



+----+--------------+--------------+
|  yr|max(Adj Close)|min(Adj Close)|
+----+--------------+--------------+
|2010|      2.364667|      1.053333|
|2011|      2.329333|      1.455333|
|2012|         2.534|      1.519333|
|2013|     12.891333|         2.194|
|2014|     19.069332|      9.289333|
|2015|     18.817333|     12.333333|
|2016|     17.694668|         9.578|
|2017|     25.666668|        14.466|
|2018|     25.304667|        16.704|
|2019|     28.729334|     11.931333|
|2020|    235.223328|     24.081333|
|2021|    409.970001|    187.666672|
|2022|    399.926666|    109.099998|
|2023|    293.339996|    108.099998|
|2024|     263.26001|    142.050003|
+----+--------------+--------------+



In [73]:
sqlContext.sql("SELECT year(tesla_stocks.Date) as yr, max(tesla_stocks.`Adj Close`), min(tesla_stocks.`Adj Close`) FROM tesla_stocks group By year(tesla_stocks.Date)").explain()



== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[_groupingexpression#1332], functions=[max(Adj Close#226), min(Adj Close#226)])
   +- Exchange hashpartitioning(_groupingexpression#1332, 200), ENSURE_REQUIREMENTS, [plan_id=1301]
      +- HashAggregate(keys=[_groupingexpression#1332], functions=[partial_max(Adj Close#226), partial_min(Adj Close#226)])
         +- Project [Adj Close#226, year(Date#221) AS _groupingexpression#1332]
            +- FileScan csv [Date#221,Adj Close#226] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/tequan/Downloads/stock analysis pyspark/TSLA.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Date:date,Adj Close:double>




In [74]:
# Join all stock closing prices in order to compare

joinclose=sqlContext.sql("SELECT tesla_stocks.Date, tesla_stocks.`Adj Close` as teslaclose, amazon_stocks.`Adj Close` as amazonclose, google_stocks.`Adj Close` as googleclose from tesla_stocks join google_stocks on tesla_stocks.Date = google_stocks.Date join amazon_stocks on tesla_stocks.Date = amazon_stocks.Date").cache()

joinclose.show()

joinclose.registerTempTable("joinclose")

+----------+----------+-----------+-----------+
|      Date|teslaclose|amazonclose|googleclose|
+----------+----------+-----------+-----------+
|2010-06-29|  1.592667|     5.4305| 227.357361|
|2010-06-30|  1.588667|      5.463| 222.697693|
|2010-07-01|     1.464|      5.548| 219.964966|
|2010-07-02|      1.28|      5.457|   218.4935|
|2010-07-06|     1.074|      5.503|  218.25325|
|2010-07-07|  1.053333|     5.6715| 225.325317|
|2010-07-08|     1.164|      5.811| 228.508514|
|2010-07-09|      1.16|      5.863| 233.978973|
|2010-07-12|  1.136667|     5.9755| 238.153152|
|2010-07-13|  1.209333|     6.1825| 244.844849|
|2010-07-14|  1.322667|      6.165| 245.915909|
|2010-07-15|     1.326|      6.103| 247.257263|
|2010-07-16|     1.376|     5.9245| 230.035034|
|2010-07-19|  1.460667|      5.997| 233.323318|
|2010-07-20|  1.353333|      6.005| 241.036041|
|2010-07-21|     1.348|     5.8715| 238.988983|
|2010-07-22|       1.4|     6.0035| 242.647644|
|2010-07-23|  1.419333|     5.9435| 245.

In [76]:
sqlContext.sql("""
               SELECT 
                    year(joinclose.Date) as yr, 
                    avg(joinclose.teslaclose) as teslaclose, 
                    avg(joinclose.amazonclose) as amazonclose, 
                    avg(joinclose.googleclose) as googleclose 
               from joinclose 
               group By year(joinclose.Date) 
               order by year(joinclose.Date)
               """).show()



+----+------------------+------------------+------------------+
|  yr|        teslaclose|       amazonclose|       googleclose|
+----+------------------+------------------+------------------+
|2010|  1.55612306923077| 7.467653846153843|267.62454727692307|
|2011| 1.786984130952381| 9.833601190476193|284.77185524206357|
|2012| 2.077906639999998|11.015182000000001| 321.7299694959999|
|2013| 6.960082000000001|14.901579400793652| 442.5639325158732|
|2014|14.888605805555553|16.627551603174613|  568.122366142857|
|2015|15.336193071428559|23.906914666666676| 619.9882132103176|
|2016|13.984484146825396| 34.97615664285715|  763.213214210318|
|2017|20.954419609561768|48.408350561753004| 939.7730703545823|
|2018|  21.1539946693227| 82.08630851792826|1122.0435856772904|
|2019| 18.23534657539683| 89.45946026190477|1191.2169417142854|
|2020| 96.66568879446635|134.04275452569178|1478.9931240158096|
|2021| 259.9981617103175|167.19334932142868| 2484.497738793653|
|2022|307.36414908771934|153.46636861403

## **Saving to Parquet File**

In [77]:
joinclose.write.format("parquet").save("joinstocks.parquet")

In [78]:
final_df = sqlContext.read.parquet("joinstocks.parquet")
final_df.show()

+----------+----------+-----------+-----------+
|      Date|teslaclose|amazonclose|googleclose|
+----------+----------+-----------+-----------+
|2010-06-29|  1.592667|     5.4305| 227.357361|
|2010-06-30|  1.588667|      5.463| 222.697693|
|2010-07-01|     1.464|      5.548| 219.964966|
|2010-07-02|      1.28|      5.457|   218.4935|
|2010-07-06|     1.074|      5.503|  218.25325|
|2010-07-07|  1.053333|     5.6715| 225.325317|
|2010-07-08|     1.164|      5.811| 228.508514|
|2010-07-09|      1.16|      5.863| 233.978973|
|2010-07-12|  1.136667|     5.9755| 238.153152|
|2010-07-13|  1.209333|     6.1825| 244.844849|
|2010-07-14|  1.322667|      6.165| 245.915909|
|2010-07-15|     1.326|      6.103| 247.257263|
|2010-07-16|     1.376|     5.9245| 230.035034|
|2010-07-19|  1.460667|      5.997| 233.323318|
|2010-07-20|  1.353333|      6.005| 241.036041|
|2010-07-21|     1.348|     5.8715| 238.988983|
|2010-07-22|       1.4|     6.0035| 242.647644|
|2010-07-23|  1.419333|     5.9435| 245.

In [79]:
final_df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- teslaclose: double (nullable = true)
 |-- amazonclose: double (nullable = true)
 |-- googleclose: double (nullable = true)



In [80]:
# Average of each by month
final_df.select(year("Date").alias("year"), 
                 month("Date").alias("month"), 
                 "teslaclose", "amazonclose", 
                 "googleclose").groupby("year", 
                                        "month").avg("teslaclose", 
                                                     "amazonclose", 
                                                     "googleclose").sort("year", 
                                                                         "month").show()
                

+----+-----+------------------+------------------+------------------+
|year|month|   avg(teslaclose)|  avg(amazonclose)|  avg(googleclose)|
+----+-----+------------------+------------------+------------------+
|2010|    6|          1.590667|           5.44675|        225.027527|
|2010|    7|1.3038729999999998|  5.87595238095238| 236.7198143809524|
|2010|    8|1.3007272727272727| 6.318636363636364| 239.8964870909091|
|2010|    9|1.3826984285714283| 7.383214285714287|247.51489690476188|
|2010|   10|1.3808888095238092| 8.037071428571428| 288.2575436666666|
|2010|   11| 1.936126952380952| 8.426071428571431| 301.3067815714286|
|2010|   12|2.0192424545454544| 8.941954545454546| 296.0742549545455|
|2011|    1|1.7095334000000002|          9.117625|308.03428664999996|
|2011|    2| 1.555578947368421| 9.055894736842106|308.72793899999994|
|2011|    3|1.5805797826086954| 8.440195652173912|291.41859369565213|
|2011|    4|        1.74439995| 9.228825000000002|277.55955654999997|
|2011|    5|1.840857

In [81]:
final_df.select(year("Date").alias("year"), 
                 month("Date").alias("month"), 
                 "teslaclose", "amazonclose", 
                 "googleclose").groupby("year", 
                                        "month").avg("teslaclose", 
                                                     "amazonclose", 
                                                     "googleclose").sort("year", 
                                                                         "month").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [year#1963 ASC NULLS FIRST, month#1964 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(year#1963 ASC NULLS FIRST, month#1964 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=1792]
      +- HashAggregate(keys=[year#1963, month#1964], functions=[avg(teslaclose#1870), avg(amazonclose#1871), avg(googleclose#1872)])
         +- Exchange hashpartitioning(year#1963, month#1964, 200), ENSURE_REQUIREMENTS, [plan_id=1789]
            +- HashAggregate(keys=[year#1963, month#1964], functions=[partial_avg(teslaclose#1870), partial_avg(amazonclose#1871), partial_avg(googleclose#1872)])
               +- Project [year(Date#1869) AS year#1963, month(Date#1869) AS month#1964, teslaclose#1870, amazonclose#1871, googleclose#1872]
                  +- FileScan parquet [Date#1869,teslaclose#1870,amazonclose#1871,googleclose#1872] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/Users/te